In [129]:
import random

In [130]:
DILITHIUM_Q = 8380417 # 2**23 - 2**13 + 1
DILITHIUM_N = 256
DILITHIUM_K = 8
DILITHIUM_L = 7
DILITHIUM_ETA = 2
DILITHIUM_D = 13

In [131]:
def sk_decode(sk_bytes):
    poly_s_packed_len = (DILITHIUM_ETA + 1) * DILITHIUM_N // 8
    poly_t0_packed_len = DILITHIUM_D * DILITHIUM_N // 8
    s1_len = DILITHIUM_L * poly_s_packed_len
    s2_len = DILITHIUM_K * poly_s_packed_len
    t0_len = DILITHIUM_K * poly_t0_packed_len

    if len(sk_bytes) != 4*32 + s1_len + s2_len + t0_len:
        raise ValueError("SK packed bytes is of the wrong length")
    
    # Split bytes between seeds and vectors
    sk_seed_bytes, sk_vec_bytes = sk_bytes[:128], sk_bytes[128:]
    
    # Unpack seed bytes
    rho, K, tr = sk_seed_bytes[:32], sk_seed_bytes[32:64], sk_seed_bytes[64:128]
    
    # Unpack vector bytes
    s1_bytes = sk_vec_bytes[:s1_len]
    s2_bytes = sk_vec_bytes[s1_len:s1_len+s2_len]
    t0_bytes = sk_vec_bytes[-t0_len:]

    print(f"s1 bytes: {s1_bytes.hex()}")
    print(f"s2 bytes: {s2_bytes.hex()}")
    print(f"t0 bytes: {t0_bytes.hex()}")
    
    # Unpack bytes to vectors
    s1 = unpack_s(s1_bytes, DILITHIUM_L, poly_s_packed_len)
    s2 = unpack_s(s2_bytes, DILITHIUM_K, poly_s_packed_len)
    t0 = unpack_t0(t0_bytes, DILITHIUM_K, poly_t0_packed_len)

    return s1, s2, t0

def unpack_s(input_bytes, poly_num, packed_len):
    poly_bytes = [input_bytes[i:i+packed_len] for i in range(0, len(input_bytes), packed_len)]
    #print(poly_bytes[0])
    s = [poly_unpack_s(poly_bytes[i]) for i in range(poly_num)]
    return s

def poly_unpack_s(input_bytes):
    altered_coeffs = bit_unpack(input_bytes, DILITHIUM_ETA+1)
    #print(altered_coeffs)
    coefficients = [DILITHIUM_ETA - c for c in altered_coeffs] 
    return coefficients

def unpack_t0(input_bytes, poly_num, packed_len):
    poly_bytes = [input_bytes[i:i+packed_len] for i in range(0, len(input_bytes), packed_len)]
    #print(poly_bytes[0])
    t0 = [poly_unpack_t0(poly_bytes[i]) for i in range(poly_num)]
    return t0

def poly_unpack_t0(input_bytes):
    altered_coeffs = bit_unpack(input_bytes, DILITHIUM_D)
    #print(altered_coeffs)
    coefficients = [(1 << (DILITHIUM_D-1)) - c for c in altered_coeffs] 
    return coefficients

def bit_unpack(input_bytes, n_bits):
    r = int.from_bytes(input_bytes, 'little')
    mask = (1 << n_bits) - 1
    return [(r >> n_bits*i) & mask for i in range(DILITHIUM_N)]

In [132]:
import struct

def generate_random_bytes(length):
    return bytes([random.randint(0, 255) for _ in range(length)])

def generate_random_sk_bytes():
    # Generate random seed bytes
    rho = generate_random_bytes(32)
    K = generate_random_bytes(32)
    tr = generate_random_bytes(64)
    
    # Calculate vector lengths
    poly_s_packed_len = (DILITHIUM_ETA + 1) * DILITHIUM_N // 8
    poly_t0_packed_len = DILITHIUM_D * DILITHIUM_N // 8
    s1_len = DILITHIUM_L * poly_s_packed_len
    s2_len = DILITHIUM_K * poly_s_packed_len
    t0_len = DILITHIUM_K * poly_t0_packed_len
    
    # Generate random vector bytes
    s1_bytes = generate_random_bytes(s1_len)
    s2_bytes = generate_random_bytes(s2_len)    
    t0_bytes = generate_random_bytes(t0_len)
    
    # Concatenate seed and vector bytes
    sk_bytes = rho + K + tr + s1_bytes + s2_bytes + t0_bytes
    return sk_bytes

random_sk_bytes = generate_random_sk_bytes()
print(f"Random SK bytes: {random_sk_bytes.hex()}")

s1, s2, t0 = sk_decode(random_sk_bytes)

# Print 's1' as a matrix
print("s1:")
for row in s1:
    print(row)

# Print 's2' as a matrix
print("\ns2:")
for row in s2:
    print(row)

# Print 't0' as a matrix
print("\nt0:")
for row in t0:
    print(row)

Random SK bytes: 99626db759745012d269a5f39369cd9f04d55ab9186a057cc458fc39477f0963379ae605fa08c31d5c692f20a1cc909d906dc12ea849a44c8e2aeaf9614e1c4e0cee454daf28659dd45fddfb83f043597df0befab40d9b974a38529ac0c890e757f29904144fc27481363e0a1bf675d7ca83c2a74f4702a9bd17bdfb4a3dd87e45fed45c04c904e2ddf5cbf3cdcf26f544a7a264503aaf2d39e00d10e78e0ebdce1a8e24401f7ed6e3bd6158ee5c0c9e62226bd00f9fa1cdde30403629a283911cca2059409ab0ba6c40f4878551afe2e1becc06f9453b0750f5dd778b884c1588bd215b35ef8d46543c9de78d744d81284697a6a7e06abb1c7a984f1d4ad55a110fa9886ba9974dc74f7898b8bc6d5fbf6fee75a1918e9f71d06a9dc8df7eef736940cd68355057568d58e606b8187505a9962189224cafd0eac76463ef639774655b27cb0738e1ae0849eadbc46eb6e67729e306a95c73fe511254d3254a80ef7581cde2e8b65c1e5ffdc9f91301779afd53621cd05b3294398c9713ef71a0c6c9ecacffb6fe71591297f68898d07d61065939e636bddf8d9c09f3293216c8a7ef80ba4c8947daa1480bf2f9953e2f887f046073c00575aa9cbd3e4a4d5a0a20dbb005db50ec7f5e83b6bdcdc420c1ec06af99ad41ab6b9c1fdc9506832340965c20a570f2351ecaa7a57